In [1]:
import datetime as dt
import numpy as np
import pandas as pd

## Load original preprocessed data

In [2]:
data = pd.read_csv("../v1/preprocessed_data.csv")

# Convert "recorded" column to Python datetime objects
data["recorded"] = pd.to_datetime(data["recorded"], format="%Y-%m-%d %H:%M:%S")
data.head(len(data))

recorded   ct    ss  tsc    lane   ds1   mf1   rf1   ds2  \
0      2021-02-20 20:04:00  110  2475  203  SA-523  38.0  12.0  11.0  38.0   
1      2021-02-20 20:04:00  110  2480  221  SA-572  36.0   9.0  10.0  60.0   
2      2021-02-20 20:04:00  110  2469  181  SA-528  41.0  12.0  12.0  37.0   
3      2021-02-20 20:04:00  110  2519  364  SA-532  27.0  10.0  10.0  33.0   
4      2021-02-20 20:04:00  110  2447  278  SA-550  43.0  20.0  19.0  14.0   
...                    ...  ...   ...  ...     ...   ...   ...   ...   ...   
651286 2021-06-18 16:18:00  141  2447   88  SA-535  98.0  36.0  38.0  69.0   
651287 2021-06-18 16:18:00  141  2447  278  SA-538  55.0  32.0  30.0  25.0   
651288 2021-06-18 16:18:00  141  2447  278  SA-550  78.0  39.0  36.0  25.0   
651289 2021-06-18 16:18:00  141  2480  221  SA-572  80.0  24.0  26.0  92.0   
651290 2021-06-18 16:18:00  141  2519  364  SA-532  56.0  31.0  29.0  43.0   

         mf2   rf2   ds3   mf3   rf3  
0       12.0  11.0  19.0   6.0   5.0  
1       17.0  18.0  16.0   4.0   5.0  
2       10.0  11.0  23.0   7.0   6.0  
3       12.0  12.0  10.0   5.0   4.0  
4       11.0  10.0  10.0   8.0   7.0  
...      ...   ...   ...   ...   ...  
651286  25.0  26.0  36.0  14.0  15.0  
651287  16.0  15.0  13.0  10.0   9.0  
651288  17.0  14.0  16.0  10.0   7.0  
651289  31.0  32.0  88.0  32.0  31.0  
651290  25.0  24.0  31.0  17.0  15.0  

[651291 rows x 14 columns]

In [3]:
# Relevant TSCs linked to Coronation drive
tscs = [87, 88, 278, 181, 188, 203, 221, 364]

In [4]:
# Make "recorded" column new dataframe index
data.index = data["recorded"]
data.drop("recorded", axis=1, inplace=True)

## Several dates with missing samples usually between 5 am - 12 pm. Forward fill with samples, same time one week prior. If no data available, forward fill with samples, one day prior.

In [5]:
# Stores NaN observations 
tsc_nan_dict = {}
# New dataframe containing samples for each TSC in 5 minute intervals
df = pd.DataFrame(columns=["ct", "ss", "tsc", "ds1", "mf1", "rf1", "ds2", "mf2", "rf2", "ds3", "mf3", "rf3"])

for tsc in tscs:
    ss = data[data["tsc"] == tsc]["ss"].iloc[0]
    # Resampled samples for given TSC in 5 minute intervals (with NaNs)
    temp_df = data[data["tsc"] == tsc].drop(["ss", "tsc"], axis=1).resample("5T").mean()
    
    # Store dates with NaNs appearing
    nan_dates = []
    for date in temp_df[temp_df.isnull().any(axis=1)].index:
        if date.date() not in nan_dates:
            nan_dates.append(date.date())
            
    tsc_nan_dict[tsc] = nan_dates
    
    # Apply forward fill technique from either one week prior or if not available, one day prior
    temp_temp_df = temp_df[temp_df.isnull().any(axis=1)]
    for i in range(len(temp_temp_df)):
        values = temp_df[temp_df.index == (temp_temp_df.index[i] - dt.timedelta(weeks=1))].values
        if len(values) == 0:
            values = temp_df[temp_df.index == (temp_temp_df.index[i] - dt.timedelta(days=1))].values
            
        temp_df[temp_df.index == temp_temp_df.index[i]] = values
    
    # Same column label sequence
    temp_df.insert(1, column="tsc", value=[tsc]*len(temp_df))
    temp_df.insert(1, column="ss", value=[ss]*len(temp_df))
    
    df = pd.concat([df, temp_df])
               
tsc_nan_dict

{87: [datetime.date(2021, 2, 25),
  datetime.date(2021, 3, 3),
  datetime.date(2021, 3, 4),
  datetime.date(2021, 3, 5),
  datetime.date(2021, 3, 6),
  datetime.date(2021, 3, 7),
  datetime.date(2021, 3, 8),
  datetime.date(2021, 3, 9),
  datetime.date(2021, 3, 10),
  datetime.date(2021, 3, 11),
  datetime.date(2021, 3, 22),
  datetime.date(2021, 3, 24),
  datetime.date(2021, 3, 25),
  datetime.date(2021, 4, 6),
  datetime.date(2021, 4, 10),
  datetime.date(2021, 4, 11),
  datetime.date(2021, 4, 12),
  datetime.date(2021, 4, 19),
  datetime.date(2021, 4, 20),
  datetime.date(2021, 4, 21),
  datetime.date(2021, 4, 22),
  datetime.date(2021, 4, 23),
  datetime.date(2021, 4, 29),
  datetime.date(2021, 5, 2),
  datetime.date(2021, 5, 3),
  datetime.date(2021, 5, 4),
  datetime.date(2021, 5, 10),
  datetime.date(2021, 5, 12),
  datetime.date(2021, 5, 15),
  datetime.date(2021, 5, 16),
  datetime.date(2021, 5, 17),
  datetime.date(2021, 5, 18),
  datetime.date(2021, 5, 19),
  datetime.date(2

In [6]:
# Make index a separate column again and reset index
df.insert(0, column="recorded", value=df.index)
df.reset_index(drop=True)

# Sort dataframe according to time
df.sort_values("recorded", axis=0, inplace=True)
df.head(len(df))

recorded     ct    ss  tsc   ds1   mf1    rf1  \
2021-02-20 20:00:00 2021-02-20 20:00:00  110.0  2446   87  42.0  12.0  12.00   
2021-02-20 20:00:00 2021-02-20 20:00:00  110.0  2480  221  36.0   9.0  10.00   
2021-02-20 20:00:00 2021-02-20 20:00:00  110.0  2475  203  38.0  12.0  11.00   
2021-02-20 20:00:00 2021-02-20 20:00:00  110.0  2473  188  41.0  16.0  17.00   
2021-02-20 20:00:00 2021-02-20 20:00:00  110.0  2469  181  41.0  12.0  12.00   
...                                 ...    ...   ...  ...   ...   ...    ...   
2021-06-18 16:15:00 2021-06-18 16:15:00  140.5  2447  278  63.5  36.0  34.75   
2021-06-18 16:15:00 2021-06-18 16:15:00  140.5  2447   88  93.5  33.5  35.00   
2021-06-18 16:15:00 2021-06-18 16:15:00  140.5  2446   87  78.0  33.0  33.00   
2021-06-18 16:15:00 2021-06-18 16:15:00  140.5  2480  221  81.5  24.5  26.00   
2021-06-18 16:15:00 2021-06-18 16:15:00  140.5  2519  364  53.0  29.5  27.50   

                       ds2    mf2   rf2    ds3    mf3    rf3  
2021-02-20 20:00:00  47.00  14.00  14.0  25.00   8.00   7.00  
2021-02-20 20:00:00  60.00  17.00  18.0  16.00   4.00   5.00  
2021-02-20 20:00:00  38.00  12.00  11.0  19.00   6.00   5.00  
2021-02-20 20:00:00  39.00  15.00  15.0  24.00   8.00   9.00  
2021-02-20 20:00:00  37.00  10.00  11.0  23.00   7.00   6.00  
...                    ...    ...   ...    ...    ...    ...  
2021-06-18 16:15:00  35.75  24.25  23.5  22.25  15.25  13.75  
2021-06-18 16:15:00  69.50  25.00  25.5  48.50  19.00  19.50  
2021-06-18 16:15:00  87.00  38.00  38.0  68.00  31.00  30.00  
2021-06-18 16:15:00  83.00  28.00  28.5  89.50  31.50  31.50  
2021-06-18 16:15:00  45.00  26.50  25.0  37.50  21.00  18.50  

[271520 rows x 13 columns]

In [7]:
df.to_csv(path_or_buf="preprocessed_data_v2.csv", index=False)